In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# List of URLs of all players with active world rankings in 5 disciplines.
ms_base_url = 'https://bwf.tournamentsoftware.com/ranking/category.aspx?id=42511&category=472&C472FOC=&p={}&ps=100' # Men's Singles URL, filtered by 100 results on each page.
ws_base_url = 'https://bwf.tournamentsoftware.com/ranking/category.aspx?id=42633&category=473&C473FOC=&p={}&ps=100' # Women's Singles URL, filtered by 100 results on each page.
md_base_url = 'https://bwf.tournamentsoftware.com/ranking/category.aspx?id=42633&category=474&C474FOC=&p={}&ps=100' # Men's Doubles URL, filtered by 100 results on each page.
wd_base_url = 'https://bwf.tournamentsoftware.com/ranking/category.aspx?id=42633&category=475&C475FOC=&p={}&ps=100' # Women's Doubles URL, filtered by 100 results on each page.
xd_base_url = 'https://bwf.tournamentsoftware.com/ranking/category.aspx?id=42633&category=476&C476FOC=&p={}&ps=100' # Mixed Doubles URL, filtered by 100 results on each page.

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [5]:
# Create a list of column names for our DataFrame, following the HTLM inspection of the number of values in each row which is 11.
# x1 & x2 are placeholder columns which will be dropped later.
list = ['Rank', 'x1', 'x2', 'Country Code', 'Player', 'Profile', 'Member ID', 'Points', 'Tournaments', 'Confederation', 'Country']

In [6]:
# Create DataFrames for all categories. 
ms_df = pd.DataFrame(columns = list)
ws_df = pd.DataFrame(columns = list)
md_df = pd.DataFrame(columns = list)
wd_df = pd.DataFrame(columns = list)
xd_df = pd.DataFrame(columns = list)

### **Men's Singles**

In [7]:
# Filter the page by 100 results on each page. There are a total of 19 pages and 1831 results.
# We need this information to work with the last page which contains less than 100 results compared to the previous pages.
ms_page_number = 19
ms_results = 1831

# Create a loop to add data to all pages in DataFrame
for ms_page in range(1, ms_page_number + 1):
    if ms_page < ms_page_number:  # Pages 1-18 with 100 results on each page
        ms_url = ms_base_url.format(ms_page)
        response = requests.get(ms_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:102]  # Skip the header and bottom rows

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5:  # The 6th column is 'Profile' containing profile links under 'href' tags, it could be more than 2 links
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls]) # Complete the urls
                    individual_row_data.append(complete_urls)
                elif idx == 6:  # The 7th column is 'Member ID'
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts) # Split Member IDs by '/' if more than 1
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            ms_df_length = len(ms_df)
            ms_df.loc[ms_df_length] = individual_row_data # Add rows into DataFrame
            
            if len(ms_df) >= ms_results: # Prevent duplicating data when rerun the code
                break
    else:  # Last page with less than 100 results
        ms_url = ms_base_url.format(ms_page)
        response = requests.get(ms_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:(ms_results - ((ms_page_number - 1) * 100) + 2)]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5:  # The 6th column is 'Profile' containing profile links under 'href' tags, it could be more than 2 links
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls]) # Complete the urls
                    individual_row_data.append(complete_urls)
                elif idx == 6:  # The 7th column is 'Member ID'
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts) # Split Member IDs by '/' if more than 1
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            ms_df_length = len(ms_df)
            ms_df.loc[ms_df_length] = individual_row_data # Add rows into DataFrame

            if len(ms_df) >= ms_results: # Prevent duplicating data when rerun the code
                break

ms_df = ms_df.drop(columns=['x1', 'x2']) # Dropping the columns 'x1', 'x2'
ms_df['Category'] = 'MS' # Add 'Category' column
ms_df # Display the resulting DataFrame

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHN,[CHN] SHI Yu Qi,https://bwf.tournamentsoftware.com/player-prof...,57945,101815,18,Asia,China,MS
1,2,DEN,[DEN] Viktor AXELSEN,https://bwf.tournamentsoftware.com/player-prof...,25831,93790,14,Europe,Denmark,MS
2,3,DEN,[DEN] Anders ANTONSEN,https://bwf.tournamentsoftware.com/player-prof...,91554,84241,22,Europe,Denmark,MS
3,4,JPN,[JPN] Kodai NARAOKA,https://bwf.tournamentsoftware.com/player-prof...,62063,84197,22,Asia,Japan,MS
4,5,MAS,[MAS] LEE Zii Jia,https://bwf.tournamentsoftware.com/player-prof...,81561,83716,22,Asia,Malaysia,MS
...,...,...,...,...,...,...,...,...,...,...
1826,1767,UGA,[UGA] Hassan Jamil MAKINDA,https://bwf.tournamentsoftware.com/player-prof...,33916,20,1,Africa,Uganda,MS
1827,1767,UGA,[UGA] Latif NGOBI,https://bwf.tournamentsoftware.com/player-prof...,29626,20,1,Africa,Uganda,MS
1828,1767,UGA,[UGA] Abaasi BBAALE,https://bwf.tournamentsoftware.com/player-prof...,89938,20,1,Africa,Uganda,MS
1829,1767,UGA,[UGA] Preet Nikulkumar PATEL,https://bwf.tournamentsoftware.com/player-prof...,34706,20,1,Africa,Uganda,MS


### **Women's Singles**

In [8]:
# All steps are similar to the Men's Singles

ws_page_number = 13
ws_results = 1224

for ws_page in range(1, ws_page_number + 1):
    if ws_page < ws_page_number: 
        ws_url = ws_base_url.format(ws_page)
        response = requests.get(ws_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:102] 

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5: 
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6:  
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            ws_df_length = len(ws_df)
            ws_df.loc[ws_df_length] = individual_row_data
            
            if len(ws_df) >= ws_results:
                break
    else:
        ws_url = ws_base_url.format(ws_page)
        response = requests.get(ws_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:(ws_results - ((ws_page_number - 1) * 100) + 2)]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5: 
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6: 
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            ws_df_length = len(ws_df)
            ws_df.loc[ws_df_length] = individual_row_data

            if len(ws_df) >= ws_results:
                break

ws_df = ws_df.drop(columns=['x1', 'x2'])
ws_df['Category'] = 'WS'
ws_df

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHN,[CHN] CHEN Yu Fei,https://bwf.tournamentsoftware.com/player-prof...,78778,101682,14,Asia,China,WS
1,2,KOR,[KOR] AN Se Young,https://bwf.tournamentsoftware.com/player-prof...,87442,100337,12,Asia,Korea,WS
2,3,CHN,[CHN] WANG Zhi Yi,https://bwf.tournamentsoftware.com/player-prof...,61854,90895,19,Asia,China,WS
3,4,ESP,[ESP] Carolina MARIN,https://bwf.tournamentsoftware.com/player-prof...,18228,87736,14,Europe,Spain,WS
4,5,TPE,[TPE] TAI Tzu Ying,https://bwf.tournamentsoftware.com/player-prof...,61427,87111,17,Asia,Chinese Taipei,WS
...,...,...,...,...,...,...,...,...,...,...
1219,1194,DEN,[DEN] Mathilde SLOTSAGER,https://bwf.tournamentsoftware.com/player-prof...,76549,20,1,Europe,Denmark,WS
1220,1194,EST,[EST] Karolina PINTŠUK,https://bwf.tournamentsoftware.com/player-prof...,87914,20,1,Europe,Estonia,WS
1221,1194,FRA,[FRA] Manon HEITZMANN,https://bwf.tournamentsoftware.com/player-prof...,99557,20,1,Europe,France,WS
1222,1194,DEN,[DEN] Julie BRINCH,https://bwf.tournamentsoftware.com/player-prof...,98323,20,1,Europe,Denmark,WS


### **Men's Doubles**

In [9]:
# All steps are similar to the Men's Singles

md_page_number = 15
md_results = 1429

for md_page in range(1, md_page_number + 1):
    if md_page < md_page_number:
        md_url = md_base_url.format(md_page)
        response = requests.get(md_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:102]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5: 
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6:
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            md_df_length = len(md_df)
            md_df.loc[md_df_length] = individual_row_data
            
            if len(md_df) >= md_results:
                break
    else:
        md_url = md_base_url.format(md_page)
        response = requests.get(md_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:(md_results - ((md_page_number - 1) * 100) + 2)]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5:
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6:
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            md_df_length = len(md_df)
            md_df.loc[md_df_length] = individual_row_data

            if len(md_df) >= md_results:
                break

md_df = md_df.drop(columns=['x1', 'x2'])
md_df['Category'] = 'MD'
md_df

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHNCHN,[CHN] LIANG Wei Keng[CHN] WANG Chang,https://bwf.tournamentsoftware.com/player-prof...,90531/55414,96798,14,Asia,China,MD
1,2,DENDEN,[DEN] Kim ASTRUP[DEN] Anders Skaarup RASMUSSEN,https://bwf.tournamentsoftware.com/player-prof...,92980/44414,91753,20,Europe,Denmark,MD
2,3,KORKOR,[KOR] KANG Min Hyuk[KOR] SEO Seung Jae,https://bwf.tournamentsoftware.com/player-prof...,77262/66513,90415,20,Asia,Korea,MD
3,4,MASMAS,[MAS] Aaron CHIA[MAS] SOH Wooi Yik,https://bwf.tournamentsoftware.com/player-prof...,56203/99389,84315,16,Asia,Malaysia,MD
4,5,CHNCHN,[CHN] HE Ji Ting[CHN] REN Xiang Yu,https://bwf.tournamentsoftware.com/player-prof...,59673/84812,83961,23,Asia,China,MD
...,...,...,...,...,...,...,...,...,...,...
1424,1415,ESPESP,[ESP] Fernando CIVICO[ESP] Ricardo RETTIG,https://bwf.tournamentsoftware.com/player-prof...,96329/69882,20,1,Europe,Spain,MD
1425,1426,BELBEL,[BEL] Elias BRACKE[BEL] Senne HOUTHOOFD,https://bwf.tournamentsoftware.com/player-prof...,67251/55858,15,1,Europe,Belgium,MD
1426,1427,BELBEL,[BEL] Charles FOUYN[BEL] Baptiste ROLIN,https://bwf.tournamentsoftware.com/player-prof...,99703/96561,2,1,Europe,Belgium,MD
1427,1427,FIJFIJ,[FIJ] Jared CHUNG[FIJ] Robert LOO,https://bwf.tournamentsoftware.com/player-prof...,82672/55618,2,1,Oceania,Fiji,MD


### **Women's Doubles**

In [10]:
# All steps are similar to the Men's Singles

wd_page_number = 11
wd_results = 1029

for wd_page in range(1, wd_page_number + 1):
    if wd_page < wd_page_number:
        wd_url = wd_base_url.format(wd_page)
        response = requests.get(wd_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:102]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5:
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6:
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            wd_df_length = len(wd_df)
            wd_df.loc[wd_df_length] = individual_row_data
            
            if len(wd_df) >= wd_results:
                break
    else:
        wd_url = wd_base_url.format(wd_page)
        response = requests.get(wd_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:(wd_results - ((wd_page_number - 1) * 100) + 2)]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5:  # The 6th column is 'Profile'
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6:
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            wd_df_length = len(wd_df)
            wd_df.loc[wd_df_length] = individual_row_data

            if len(wd_df) >= wd_results:
                break

wd_df = wd_df.drop(columns=['x1', 'x2'])
wd_df['Category'] = 'WD'
wd_df

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHNCHN,[CHN] CHEN Qing Chen[CHN] JIA Yi Fan,https://bwf.tournamentsoftware.com/player-prof...,94125/65144,104406,13,Asia,China,WD
1,2,CHNCHN,[CHN] LIU Sheng Shu[CHN] TAN Ning,https://bwf.tournamentsoftware.com/player-prof...,81599/59880,99396,21,Asia,China,WD
2,3,KORKOR,[KOR] BAEK Ha Na[KOR] LEE So Hee,https://bwf.tournamentsoftware.com/player-prof...,56706/91292,98116,15,Asia,Korea,WD
3,4,JPNJPN,[JPN] Nami MATSUYAMA[JPN] Chiharu SHIDA,https://bwf.tournamentsoftware.com/player-prof...,65212/68282,91175,18,Asia,Japan,WD
4,5,CHNCHN,[CHN] ZHANG Shu Xian[CHN] ZHENG Yu,https://bwf.tournamentsoftware.com/player-prof...,85914/85331,81491,14,Asia,China,WD
...,...,...,...,...,...,...,...,...,...,...
1024,1023,SVKSVK,[SVK] Johanka IVANOVICOVA[SVK] Olivia KADLECOVA,https://bwf.tournamentsoftware.com/player-prof...,77605/83887,40,1,Europe,Slovakia,WD
1025,1026,TURTUR,[TUR] Yasemen BEKTAS[TUR] Cansu ERCETIN,https://bwf.tournamentsoftware.com/player-prof...,86621/91818,2,1,Europe,Türkiye,WD
1026,1026,SUISUI,[SUI] Milena SCHNIDER[SUI] Jenjira STADELMANN,https://bwf.tournamentsoftware.com/player-prof...,97529/59566,2,1,Europe,Switzerland,WD
1027,1026,PYFPYF,[PYF] Heirautea CURET[PYF] Esther TAU,https://bwf.tournamentsoftware.com/player-prof...,91906/87972,2,1,Oceania,Tahiti,WD


### **Mixed Doubles**

In [11]:
# All steps are similar to the Men's Singles

xd_page_number = 15
xd_results = 1402

for xd_page in range(1, xd_page_number + 1):
    if xd_page < xd_page_number:
        xd_url = xd_base_url.format(xd_page)
        response = requests.get(xd_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:102]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5:
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6:
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            xd_df_length = len(xd_df)
            xd_df.loc[xd_df_length] = individual_row_data
            
            if len(xd_df) >= xd_results:
                break
    else:
        xd_url = xd_base_url.format(xd_page)
        response = requests.get(xd_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_table = soup.find('table', class_='ruler')
        all_rows = ranking_table.find_all('tr')[2:(xd_results - ((xd_page_number - 1) * 100) + 2)]

        for row in all_rows:
            row_data = row.find_all('td')
            individual_row_data = []

            for idx, data in enumerate(row_data):
                if idx == 5:
                    urls = [a['href'] for a in data.find_all('a', href=True)]
                    complete_urls = ', '.join(['https://bwf.tournamentsoftware.com' + url for url in urls])
                    individual_row_data.append(complete_urls)
                elif idx == 6:
                    texts = [text for text in data.stripped_strings]
                    concatenated_texts = '/'.join(texts)
                    individual_row_data.append(concatenated_texts)
                else:
                    individual_row_data.append(data.text.strip())

            xd_df_length = len(xd_df)
            xd_df.loc[xd_df_length] = individual_row_data

            if len(xd_df) >= xd_results:
                break

xd_df = xd_df.drop(columns=['x1', 'x2'])
xd_df['Category'] = 'XD'
xd_df

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHNCHN,[CHN] ZHENG Si Wei[CHN] HUANG Ya Qiong,https://bwf.tournamentsoftware.com/player-prof...,83046/63168,106850,13,Asia,China,XD
1,2,CHNCHN,[CHN] FENG Yan Zhe[CHN] HUANG Dong Ping,https://bwf.tournamentsoftware.com/player-prof...,65267/89426,99100,16,Asia,China,XD
2,3,CHNCHN,[CHN] JIANG Zhen Bang[CHN] WEI Ya Xin,https://bwf.tournamentsoftware.com/player-prof...,63029/57246,94050,19,Asia,China,XD
3,4,KORKOR,[KOR] SEO Seung Jae[KOR] CHAE Yu Jung,https://bwf.tournamentsoftware.com/player-prof...,66513/93074,83220,14,Asia,Korea,XD
4,5,JPNJPN,[JPN] Yuta WATANABE[JPN] Arisa IGARASHI,https://bwf.tournamentsoftware.com/player-prof...,58240/79558,80970,14,Asia,Japan,XD
...,...,...,...,...,...,...,...,...,...,...
1397,1376,ISRMLT,[ISR] Mark ARONCHIK[MLT] Emily ABELA,https://bwf.tournamentsoftware.com/player-prof...,69820/91376,20,1,Europe,IsraelMalta,XD
1398,1376,ESTEST,[EST] Dennis KUMAR[EST] Mia SAKARIAS,https://bwf.tournamentsoftware.com/player-prof...,85645/80515,20,1,Europe,Estonia,XD
1399,1376,DENDEN,[DEN] Victor Roed SKOV[DEN] LæRke Wiktoria WENDEL,https://bwf.tournamentsoftware.com/player-prof...,78437/69090,20,1,Europe,Denmark,XD
1400,1376,MLTMLT,[MLT] Jeremy Mark GATT[MLT] Martina CLARK,https://bwf.tournamentsoftware.com/player-prof...,36387/68347,20,1,Europe,Malta,XD


In [12]:
# Compile all data of 5 categories into one DataFrame

# List of DataFrames to concatenate
dfs = [ms_df, ws_df, md_df, wd_df, xd_df]

# Concatenate the DataFrames
all_players = pd.concat(dfs, ignore_index=True)

# Display the resulting DataFrame
all_players

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHN,[CHN] SHI Yu Qi,https://bwf.tournamentsoftware.com/player-prof...,57945,101815,18,Asia,China,MS
1,2,DEN,[DEN] Viktor AXELSEN,https://bwf.tournamentsoftware.com/player-prof...,25831,93790,14,Europe,Denmark,MS
2,3,DEN,[DEN] Anders ANTONSEN,https://bwf.tournamentsoftware.com/player-prof...,91554,84241,22,Europe,Denmark,MS
3,4,JPN,[JPN] Kodai NARAOKA,https://bwf.tournamentsoftware.com/player-prof...,62063,84197,22,Asia,Japan,MS
4,5,MAS,[MAS] LEE Zii Jia,https://bwf.tournamentsoftware.com/player-prof...,81561,83716,22,Asia,Malaysia,MS
...,...,...,...,...,...,...,...,...,...,...
6910,1376,ISRMLT,[ISR] Mark ARONCHIK[MLT] Emily ABELA,https://bwf.tournamentsoftware.com/player-prof...,69820/91376,20,1,Europe,IsraelMalta,XD
6911,1376,ESTEST,[EST] Dennis KUMAR[EST] Mia SAKARIAS,https://bwf.tournamentsoftware.com/player-prof...,85645/80515,20,1,Europe,Estonia,XD
6912,1376,DENDEN,[DEN] Victor Roed SKOV[DEN] LæRke Wiktoria WENDEL,https://bwf.tournamentsoftware.com/player-prof...,78437/69090,20,1,Europe,Denmark,XD
6913,1376,MLTMLT,[MLT] Jeremy Mark GATT[MLT] Martina CLARK,https://bwf.tournamentsoftware.com/player-prof...,36387/68347,20,1,Europe,Malta,XD


### **Cleaning data**

In [13]:
# Cleaning the 'Country Code' column. For Doubles categories, we need to split the country codes of 2 players by '/', which is under the 6-character format. 

# Create a function to format the 6-character values
def format_category(value):
    if len(value) == 6:
        return value[:3] + '/' + value[3:]
    return value

# Apply the function to the 'Country Code' column
all_players['Country Code'] = all_players['Country Code'].apply(format_category)

# Display the resulting DataFrame
all_players

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHN,[CHN] SHI Yu Qi,https://bwf.tournamentsoftware.com/player-prof...,57945,101815,18,Asia,China,MS
1,2,DEN,[DEN] Viktor AXELSEN,https://bwf.tournamentsoftware.com/player-prof...,25831,93790,14,Europe,Denmark,MS
2,3,DEN,[DEN] Anders ANTONSEN,https://bwf.tournamentsoftware.com/player-prof...,91554,84241,22,Europe,Denmark,MS
3,4,JPN,[JPN] Kodai NARAOKA,https://bwf.tournamentsoftware.com/player-prof...,62063,84197,22,Asia,Japan,MS
4,5,MAS,[MAS] LEE Zii Jia,https://bwf.tournamentsoftware.com/player-prof...,81561,83716,22,Asia,Malaysia,MS
...,...,...,...,...,...,...,...,...,...,...
6910,1376,ISR/MLT,[ISR] Mark ARONCHIK[MLT] Emily ABELA,https://bwf.tournamentsoftware.com/player-prof...,69820/91376,20,1,Europe,IsraelMalta,XD
6911,1376,EST/EST,[EST] Dennis KUMAR[EST] Mia SAKARIAS,https://bwf.tournamentsoftware.com/player-prof...,85645/80515,20,1,Europe,Estonia,XD
6912,1376,DEN/DEN,[DEN] Victor Roed SKOV[DEN] LæRke Wiktoria WENDEL,https://bwf.tournamentsoftware.com/player-prof...,78437/69090,20,1,Europe,Denmark,XD
6913,1376,MLT/MLT,[MLT] Jeremy Mark GATT[MLT] Martina CLARK,https://bwf.tournamentsoftware.com/player-prof...,36387/68347,20,1,Europe,Malta,XD


In [14]:
# Cleaning the 'Player' column. We need to split 2 players' names by '/' for Doubles categories, then remove the country codes which are unimportant. 

import re
# Create a function to process the 'Player' column
def format_player(value):
    # Add '/' before the second country code
    value = re.sub(r'(\[\w{3}\])([^[]+)(\[\w{3}\])', r'\1\2/\3', value)
    # Remove all occurrences of country codes
    value = re.sub(r'\[\w{3}\]', '', value)
    # Remove any leading/trailing whitespace
    return value.strip()

# Apply the function to the 'Player' column
all_players['Player'] = all_players['Player'].apply(format_player)

# Display the resulting DataFrame
all_players

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHN,SHI Yu Qi,https://bwf.tournamentsoftware.com/player-prof...,57945,101815,18,Asia,China,MS
1,2,DEN,Viktor AXELSEN,https://bwf.tournamentsoftware.com/player-prof...,25831,93790,14,Europe,Denmark,MS
2,3,DEN,Anders ANTONSEN,https://bwf.tournamentsoftware.com/player-prof...,91554,84241,22,Europe,Denmark,MS
3,4,JPN,Kodai NARAOKA,https://bwf.tournamentsoftware.com/player-prof...,62063,84197,22,Asia,Japan,MS
4,5,MAS,LEE Zii Jia,https://bwf.tournamentsoftware.com/player-prof...,81561,83716,22,Asia,Malaysia,MS
...,...,...,...,...,...,...,...,...,...,...
6910,1376,ISR/MLT,Mark ARONCHIK/ Emily ABELA,https://bwf.tournamentsoftware.com/player-prof...,69820/91376,20,1,Europe,IsraelMalta,XD
6911,1376,EST/EST,Dennis KUMAR/ Mia SAKARIAS,https://bwf.tournamentsoftware.com/player-prof...,85645/80515,20,1,Europe,Estonia,XD
6912,1376,DEN/DEN,Victor Roed SKOV/ LæRke Wiktoria WENDEL,https://bwf.tournamentsoftware.com/player-prof...,78437/69090,20,1,Europe,Denmark,XD
6913,1376,MLT/MLT,Jeremy Mark GATT/ Martina CLARK,https://bwf.tournamentsoftware.com/player-prof...,36387/68347,20,1,Europe,Malta,XD


In [15]:
# Cleaning the 'Country' column. We need to split 2 countries by '/' for Doubles categories. 

country_list = all_players['Country'].tolist()

# Create a function to separate concatenated country names
def separate_countries(value):
    for country in country_list:
        if value.startswith(country):
            rest = value[len(country):]
            if rest and any(rest.startswith(other_country) for other_country in country_list):
                return country + '/' + rest
    return value

# Apply the function to the 'Country' column
all_players['Country'] = all_players['Country'].apply(separate_countries)

# Display the resulting DataFrame
all_players

,Rank,Country Code,Player,Profile,Member ID,Points,Tournaments,Confederation,Country,Category
0,1,CHN,SHI Yu Qi,https://bwf.tournamentsoftware.com/player-prof...,57945,101815,18,Asia,China,MS
1,2,DEN,Viktor AXELSEN,https://bwf.tournamentsoftware.com/player-prof...,25831,93790,14,Europe,Denmark,MS
2,3,DEN,Anders ANTONSEN,https://bwf.tournamentsoftware.com/player-prof...,91554,84241,22,Europe,Denmark,MS
3,4,JPN,Kodai NARAOKA,https://bwf.tournamentsoftware.com/player-prof...,62063,84197,22,Asia,Japan,MS
4,5,MAS,LEE Zii Jia,https://bwf.tournamentsoftware.com/player-prof...,81561,83716,22,Asia,Malaysia,MS
...,...,...,...,...,...,...,...,...,...,...
6910,1376,ISR/MLT,Mark ARONCHIK/ Emily ABELA,https://bwf.tournamentsoftware.com/player-prof...,69820/91376,20,1,Europe,Israel/Malta,XD
6911,1376,EST/EST,Dennis KUMAR/ Mia SAKARIAS,https://bwf.tournamentsoftware.com/player-prof...,85645/80515,20,1,Europe,Estonia,XD
6912,1376,DEN/DEN,Victor Roed SKOV/ LæRke Wiktoria WENDEL,https://bwf.tournamentsoftware.com/player-prof...,78437/69090,20,1,Europe,Denmark,XD
6913,1376,MLT/MLT,Jeremy Mark GATT/ Martina CLARK,https://bwf.tournamentsoftware.com/player-prof...,36387/68347,20,1,Europe,Malta,XD


In [16]:
# Export the result as a csv file.
all_players.to_csv(r'C:\Users\Minh.Nguyen\OneDrive - Stats Perform\_badminton - WORLD - BWF\Badminton Player List.csv', index = False)